In [1]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm
import json

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

def file_to_string(fileName,start,end):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string.replace('$startDate$',start).replace('$endDate$',end)

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def add_conditionals(df):
    df['Date']=pd.to_datetime(df['Date'])
    test_group = np.array(['364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275']).astype(int)
    df['TestGroup'] = (df['Employee'].isin((test_group))).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Constant'] =1
    return df
    
def overlaid_histogram(df, seriesNameList, title, x_axisTitle, histnorm='',bins=100):
    import plotly.graph_objects as go
    # Build histograms
    df = df.copy()
    fig = go.Figure()
    for series in seriesNameList:
        fig.add_trace(go.Histogram(x=df[series],histnorm=histnorm,name=series, nbinsx=bins ))

    # Overlay histograms
    fig.update_layout(
        barmode='overlay',
        xaxis_title_text=x_axisTitle,
        yaxis_title_text=histnorm,
        title = title)
    # Reduce opacity to see both histograms
    fig.update_traces(opacity=0.6)
    fig.show()

# Run query
Uses a fuzzy join to see how many helix searchs/expert assist messages an expert had durring the time where they were taking a call.

In [2]:
dfp = run_query(
    file_to_string(
        fileName='AHT And Sales - With Helix and Expert Assist Fuzzy Join.SQL',
        start='2023-09-01',
        end = '2023-10-19'
    ))
display(dfp.describe(),dfp.head())

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_6304\3233110593.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,Employee,HandleTime,Offered,Accepted,HelixSearches,ExpertAssistSearches
count,"14,867.00","14,867.00","14,867.00","14,867.00","14,867.00","14,867.00"
mean,"564,579.95","1,114.49",0.61,0.07,0.11,0.18
std,"33,626.73",945.56,0.49,0.25,0.52,0.92
mad (mean),"16,576.00",677.52,0.47,0.13,0.21,0.33
min,"293,932.00",2.00,0.00,0.00,0.00,0.00
25%,"567,519.00",482.00,0.00,0.00,0.00,0.00
50%,"573,192.00",860.00,1.00,0.00,0.00,0.00
75%,"578,299.00","1,457.50",1.00,0.00,0.00,0.00
max,"581,694.00","11,253.00",1.00,1.00,10.00,26.00


,Date,Employee,TaskQueue,HandleTime,Offered,Accepted,HelixSearches,ExpertAssistSearches
0,2023-09-21,568568,VZN_MTS_Bundle_EN_2233,1085,1,0,0,0
1,2023-09-26,568568,VZN_MTS_Bundle_EN_2233,9,1,0,0,0
2,2023-09-12,568568,VZN_MTS_Bundle_EN_2233,411,1,0,0,0
3,2023-10-10,568568,VZN_Onboarding_2628,1049,1,0,0,0
4,2023-09-18,568568,VZN_MTS_Bundle_EN_2233,1673,1,0,0,0


In [3]:
df = dfp.copy()
conds = [np.logical_and(df['HelixSearches']>=1,df['ExpertAssistSearches']>=1),df['ExpertAssistSearches']>=1,df['HelixSearches']>=1]
choices=['Both','Expert Assist','Helix']
df['Tool Used'] = np.select(condlist=conds,choicelist=choices,default='None')
# display(df.head())
percentile_List = [.1,.2,.3,.4,.5,.6,.7,.8,.9] #[.5,.75,.9,.95]
describe_df = pd.DataFrame(df['HandleTime'].describe(percentiles=percentile_List)).rename({'HandleTime':'Total With or Without Tools'},axis=1)
for tool in ['None','Expert Assist','Helix','Both']:
        describe_df=describe_df.join(pd.DataFrame(df[df['Tool Used']==tool]['HandleTime'].describe(percentiles=percentile_List)).rename({'HandleTime':tool},axis=1))
        # px.histogram(df[df['Tool Used']==tool],color='Tool Used',x='HandleTime',range_x=[0,10000],nbins=10,histnorm='percent').show()
del tool
        
        
display(describe_df)
del df, conds, choices, percentile_List, describe_df

,Total With or Without Tools,None,Expert Assist,Helix,Both
count,"14,867.00","12,919.00",930.00,961.00,57.00
mean,"1,114.49","1,031.06","1,644.98","1,681.93","1,801.00"
std,945.56,871.60,"1,129.28","1,263.36","1,140.12"
mad (mean),677.52,625.06,847.37,911.80,790.46
min,2.00,2.00,8.00,24.00,128.00
10%,251.00,227.00,556.00,522.00,855.20
20%,408.00,376.00,734.00,732.00,"1,009.00"
30%,554.80,513.40,918.80,928.00,"1,145.40"
40%,697.00,654.00,"1,139.60","1,124.00","1,408.00"
50%,860.00,800.00,"1,329.00","1,345.00","1,522.00"


In [6]:
df =dfp.copy()
df = add_conditionals(df)
print(df.columns)
# df[[
#     'Date', 'Employee', 'HandleTime', 'Offered',
#     'Accepted','HelixSearches', 'ExpertAssistSearches',
#     'TestGroup','AfterExpertAssistLaunch','TaskQueue'
#     ]].to_excel('AHT And Sales - With Helix and Expert Assist Fuzzy Join.xlsx')
display(pd.pivot_table(df,index='TaskQueue,'))
del df

Index(['Date', 'Employee', 'TaskQueue', 'HandleTime', 'Offered', 'Accepted',
       'HelixSearches', 'ExpertAssistSearches', 'TestGroup',
       'AfterExpertAssistLaunch', 'ExpertAssistOnForThisUser', 'Constant'],
      dtype='object')


In [5]:
df = dfp.copy()
conds = [np.logical_and(df['HelixSearches']>=1,df['ExpertAssistSearches']>=1),df['ExpertAssistSearches']>=1,df['HelixSearches']>=1]
choices=['Both','Expert Assist','Helix']
df['Tool Used'] = np.select(condlist=conds,choicelist=choices,default='None')
px.strip(df[df['Tool Used'].isin(['Expert Assist','Helix'])].copy(),y='HandleTime',hover_data=['Date'],color='Tool Used').show()
px.scatter(df[df['Tool Used'].isin(['Expert Assist','Helix'])].copy(),y='HandleTime',x='Date',color='Tool Used').show()
df = df.pivot(columns='Tool Used',values='HandleTime')

overlaid_histogram(df,
    seriesNameList=['Helix','Expert Assist'],
    x_axisTitle='Handle Time',title='AHT by Tool Use',
    bins=30,
    histnorm='percent')

del df, conds, choices